In [1]:
# from google.colab import drive
# drive.mount('/content/drive')
# !cp /content/drive/MyDrive/CA4/*.py .
# !cp -r /content/drive/MyDrive/CA4/conf .

In [2]:
!pip install hydra-core --no-binary :all:
!pip install wandb

In [3]:
%load_ext autoreload
%autoreload 2
from train import run
import os
import os.path as osp
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf

In [4]:
with initialize(version_base=None, config_path="conf"):
    cfg = compose(config_name="config")
    print(cfg)
    data_root = cfg.data.data_root
    if not osp.exists(data_root):
        os.makedirs(data_root, exist_ok=True)
        data_root_father = osp.dirname(data_root)
        !wget https://www.cis.upenn.edu/~jshi/ped_html/PennFudanPed.zip -O {data_root_father}/data.zip
        !unzip {data_root_father}/data.zip -d {data_root_father}

{'output_dir': 'tmp/output/first', 'wandb': {'project': 'pedestrian-detection', 'name': 'debug'}, 'model': {'in_channels': 3, 'out_channels': 1, 'kernel_size': 3, 'main_channel': 8, 'depth': 3, 'spatial_scale_factor': 2, 'channel_scale_factor': 4}, 'data': {'data_root': 'tmp/data/PennFudanPed'}, 'train': {'batch_size': 8, 'num_epochs': 300, 'num_workers': 4, 'img_size': 128, 'dtype': 'bfloat16', 'device': 'cuda', 'seed': 42, 'optimizer': {'_target_': 'torch.optim.Adam', 'lr': 0.0001}, 'scheduler': {'_target_': 'utils.CosineAnnealingWarmupRestarts', 'total_epochs': '${train.num_epochs}', 'warmup_epochs': None, 'min_lr': None, 'max_lr': '${train.optimizer.lr}'}}}


In [5]:
# wandb
import wandb
wandb.login()


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ermu2001 (ermuzzz2001). Use `wandb login --relogin` to force relogin


True

# Define a sweep

In [6]:
sweep_configuration = {
    "name": "sweep-hyperparams",
    "method": "bayes",
    "metric": {"goal": "maximize", "name": "epoch_val_dice_score"},
    "parameters": {
        "learning_rate": {
            "values": [1e-4, 1e-5],

        },
        "batch_size": {"values": [8, 16, 32]},
        "epochs": {"values": [30, 120, 300]},
        "img_size": {
            "values": [64, 128],
        },
    },
}
# sweep_id = wandb.sweep(sweep=sweep_configuration, project="pedestrian-detection")
sweep_id = "bp8sw90v"
print(sweep_id)

bp8sw90v


# Sweep run

In [7]:
from train import run
def sweep_train():
    from hydra import initialize, compose
    wandb.init()
    with initialize(version_base=None, config_path="conf"):
        cfg = compose(config_name="config",
                overrides=[f"train.optimizer.lr={wandb.config.learning_rate}",
                      f"train.batch_size={wandb.config.batch_size}",
                      f"train.num_epochs={wandb.config.epochs}",
                      f"train.img_size={wandb.config.img_size}",
                      f"wandb.name={wandb.run.name}",])


        run(cfg)

In [8]:
wandb.agent(sweep_id, function=sweep_train, project="pedestrian-detection")

wandb: Agent Starting Run: uh2wowlw with config:
wandb: 	batch_size: 8
wandb: 	epochs: 300
wandb: 	img_size: 128
wandb: 	learning_rate: 0.0001


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


  0%|          | 0/17 [00:00<?, ?it/s]/home/zhaoyilin/miniconda3/envs/dl/lib/python3.10/site-packages/torch/autograd/__init__.py:200: UserWarning: upsample_bilinear2d_backward_out_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at ../aten/src/ATen/Context.cpp:71.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
/home/zhaoyilin/COURSES/IVP/CA4/model.py:121: UserWarning: Model directory tmp/output/first/epoch00000 already exists! Overwriting.
  warnings.warn(f"Model directory {model_dir} already exists! Overwriting.")
/home/zhaoyilin/COURSES/IVP/CA4/model.py:121: UserWarning: Model directory tmp/output/first/epoch00001 already exists! Overwriting.
  warnings.warn(f"Model directory {model_dir} already exists! 

epoch_learning_rate,▃▆▇███████▇▇▆▆▆▆▆▆▆▅▅▄▄▄▄▃▃▃▃▃▃▂▂▂▁▁▁▁▁▁
epoch_train_loss,██▅▅▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch_val_dice_loss,█▄▅▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch_val_dice_score,▁▁▅▅▄▇██████████████████████████████████
epoch_val_iou,▁▅▄▅▄▇██████████████████████████████████
epoch_learning_rate,0.0
epoch_train_loss,0.04848
epoch_val_dice_loss,0.26811
epoch_val_dice_score,0.73189
epoch_val_iou,0.7099


wandb: Agent Starting Run: wee6zyys with config:
wandb: 	batch_size: 16
wandb: 	epochs: 300
wandb: 	img_size: 128
wandb: 	learning_rate: 0.0001


...

epoch_learning_rate,▂▄▄▅▇█████▇▇▇▇▇▇▇▇▆▆▅▅▅▅▅▄▄▄▃▃▃▃▂▂▁▁▁▁▁▁
epoch_train_loss,████▇▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch_val_dice_loss,███▇▆▇▅▄▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch_val_dice_score,▁▁▆▇▇███████████████████████████████████
epoch_val_iou,▁▂▅▅▇▇██████████████████████████████████
epoch_learning_rate,0.0
epoch_train_loss,0.06293
epoch_val_dice_loss,0.28143
epoch_val_dice_score,0.71857
epoch_val_iou,0.70577


wandb: Agent Starting Run: y3ev5dmo with config:
wandb: 	batch_size: 16
wandb: 	epochs: 300
wandb: 	img_size: 128
wandb: 	learning_rate: 0.0001


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


  0%|          | 0/9 [00:00<?, ?it/s]/home/zhaoyilin/miniconda3/envs/dl/lib/python3.10/site-packages/torch/autograd/__init__.py:200: UserWarning: upsample_bilinear2d_backward_out_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at ../aten/src/ATen/Context.cpp:71.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
/home/zhaoyilin/COURSES/IVP/CA4/model.py:121: UserWarning: Model directory tmp/output/first/epoch00000 already exists! Overwriting.
  warnings.warn(f"Model directory {model_dir} already exists! Overwriting.")
/home/zhaoyilin/COURSES/IVP/CA4/model.py:121: UserWarning: Model directory tmp/output/first/epoch00001 already exists! Overwriting.
  warnings.warn(f"Model directory {model_dir} already exists! O

 56%|█████▌    | 5/9 [00:02<00:01,  2.59it/s]